<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
      Stress Detection Using Smartwatch Data <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial'><b>Introduction:</b></p>
<p style='font-size:16px;font-family:Arial'>
With rising stress levels globally, there's a growing need for scalable, continuous, and non-invasive stress monitoring solutions. Traditional methods like self-reported surveys and periodic clinical assessments are subjective, episodic, and not feasible for real-time monitoring. Smartwatches and other wearable devices are increasingly equipped with sensors capable of capturing physiological data, offering a powerful alternative. However, key challenges still remain:
<ul style='font-size:16px;font-family:Arial'>
    <li>Limited access to advanced biosensors like EDA or EMG in most consumer-grade smartwatches.</li>
    <li>High-frequency raw signal data requires extensive preprocessing and transformation for analysis.</li>
    <li>Lack of labeled data for stress events, especially in naturalistic (non-lab) settings.</li>
    <li>Difficulties in correlating physiological signals with contextual behaviors like sleep or activity levels.</li>
</ul>

<p style='font-size:16px;font-family:Arial'>
<b>Proposed Solution:</b><br>
Smartwatch-Based Stress Detection Using AI/ML and Vantage Analytics<br>
Leverage data from widely available smartwatch sensors (e.g., ECG, BVP, Accelerometer) to detect stress using machine learning models. Extract block-wise statistical features using Vantage’s in-database processing capabilities to ensure scalable and parallelized analysis. Augment stress detection by integrating sleep stage classification, enabling holistic health insights. The end-to-end solution allows for identifying high-stress intervals, correlating them with sleep and activity behavior, and ultimately enabling personalized wellness interventions.
</p>

<p style='font-size:16px;font-family:Arial'>
<b>Benefits:</b>
<ul style='font-size:16px;font-family:Arial'>
    <li>Early and continuous detection of stress using physiological signals from smartwatches.</li>
    <li>Improved individual well-being through timely recommendations and interventions.</li>
    <li>Integration of stress detection with sleep analytics to provide contextual insights.</li>
    <li>Scalable and parallel in-database feature extraction using Teradata Vantage.</li>
    <li>Support for future expansion to lifestyle-based recommendations or health apps.</li>
</ul>
</p>


<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>1. Connect to Vantage, Import python packages and explore the dataset</b></p>

In [ ]:
#import libraries
import getpass
from teradataml import *

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

display.max_rows=5

<p style = 'font-size:16px;font-family:Arial'>We will be prompted to provide the password. We will enter the password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=Smartwatch_Report.ipynb;' UPDATE FOR SESSION; ''')

<p style = 'font-size:18px;font-family:Arial'> <b>Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage. We have the option of either running the demo using foreign tables to access the data without using any storage on our environment or downloading the data to local storage, which may yield somewhat faster execution. However, we need to consider available storage. There are two statements in the following cell, and one is commented out. We may switch which mode we choose by changing the comment string.</p>

In [ ]:
# takes about 10 minutes
%run -i ../run_procedure.py "call get_data('DEMO_HealthcareSW_local');"

<p style = 'font-size:16px;font-family:Arial'>Optional step – We should execute the below step only if we want to see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../run_procedure.py "call space_report();"

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>2. Data Exploration </b></p>

In [ ]:
db_list_tables('DEMO_HealthcareSW')

<p style = 'font-size:16px;font-family:Arial'>The following datasets are used in this analysis to train, validate, and correlate stress detection:
<ul  style='font-size:16px;font-family:Arial'>
  <li><b>StressFeatures_train:</b> Processed physiological features derived from smartwatch signals.</li>
  <li><b>Predictions_on_Sleep_Data:</b> Predicted stress levels correlated with sleep metrics.</li>
  <li><b>Wesad:</b> Multimodal raw dataset used for initial feature extraction and labeling.</li>
</ul></p>

In [ ]:
df_stress = DataFrame(in_schema("DEMO_HealthcareSW", "StressFeatures_train "))
df_stress

In [ ]:
df_sleep = DataFrame(in_schema("DEMO_HealthcareSW", "Predictions_on_Sleep_Data"))
df_sleep

In [ ]:
df_wesad = DataFrame(in_schema("DEMO_HealthcareSW", "Wesad "))
df_wesad

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>3. Dashboard and Insights </b></p>


<p style='font-size:16px;font-family:Arial'>
This section showcases a visual summary of our Smartwatch Stress Detection analysis. The interactive dashboard provides insights into how stress (detected via physiological signals like heart rate and BVP) correlates with sleep behaviors such as deep sleep (SWS), insomnia, and restlessness.
</p>

<p style='font-size:16px;font-family:Arial'>
Key features of the dashboard:
<ul style='font-size:16px;font-family:Arial'>
    <li>Scatter plot showing relationship between hours spent in stress and percentage of deep sleep (Stage 3 and 4).</li>
    <li>Pie charts representing distribution of work shifts, and conditions such as sleep apnea, insomnia, and restlessness.</li>
    <li>Breakdown of subject sleep characteristics and working patterns to identify potential lifestyle factors contributing to stress.</li>
</ul>
</p>

<p style='font-size:16px;font-family:Arial'><b>Live Dashboard Link:</b>
<a href="https://superset.env.ci.clearscape.teradata.com/superset/dashboard/e7046cdf-79da-4fc3-92ce-53794185fa8d/?permalink_key=P62LM9lLmRw&standalone=true#TAB-BRrqQIMwv5yxxeF-VL19c" target="_blank">Smart Watch Dashboard</a>
</p>

<p style='font-size:16px;font-family:Arial'><b>Dashboard Snapshot:</b></p>
<img src="images/smartwatch.jpg" style="border: 4px solid #404040; border-radius: 10px;"/>

<p style='font-size:16px;font-family:Arial'><b>Insights:</b></p>
<ul style='font-size:16px;font-family:Arial'>
    <li>There is a general trend indicating that individuals with higher stress levels often experience reduced deep sleep.</li>
    <li>Majority of the subjects do not report conditions like insomnia or sleep apnea, indicating the dataset is relatively healthy but still stress-affected.</li>
    <li>Work shift variability (e.g., irregular shifts) might be contributing to disrupted sleep and elevated stress.</li>
</ul>
<p style='font-size:16px;font-family:Arial'>
Together, these visuals provide compelling evidence that smartwatch data can reliably reflect real-world stress and sleep interactions, opening pathways for personalized health monitoring.
</p>


<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial;'>4. Cleanup</b>

<p style = 'font-size:18px;font-family:Arial;'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial;'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../run_procedure.py "call remove_data('DEMO_HealthcareSW');"        # Takes 10 seconds

In [ ]:
remove_context()

<footer style="padding-bottom: 35px; background: #f9f9f9; border-bottom: 3px solid #00233C">
    <div style="float: left; margin-top: 14px;">ClearScape Analytics™</div>
    <div style="float: right;">
        <div style="float: left; margin-top: 14px;">Copyright © Teradata Corporation - 2025. All Rights Reserved.</div>
    </div>
</footer>